In [30]:
import datetime
import time
import urllib.request
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt

In [31]:
def get_df_from_yahoo_finance (symbol): 

    end_date = datetime.datetime.now()
    end_date = int(round(end_date.timestamp())) * 1000  #<-- Convert datetime to milliseconds 
    end_date_str = str(end_date)
    end_date_str = end_date_str[:10]  #<-- This removes all but the last 10 millisecond values for the time which is format yahoo wants

    start_date_str = '345427200'
  

    ticker_dataframe = {}
    

    # Build Yahoo finance URL for the data request.
    
    yahoo_finance_url_with_symbol = ('https://query1.finance.yahoo.com/v7/finance/download/' + str(symbol)
                                     + '?period1=' + str(start_date_str) + '&period2=' + str(end_date_str)
                                     + '&interval=1mo&events=history&includeAdjustedClose=true')

    url2 = urllib.request.urlopen(yahoo_finance_url_with_symbol) #request data (comes as CSV)
    
    ticker_dataframe = pd.read_csv(url2)  #convert CSV in memory from the Yahoo request to dataframe
    ticker_dataframe.insert(1, "Ticker", symbol)

    return (ticker_dataframe)

In [32]:
list_of_tickers_for_yahoo_data_dump = ['BTC-USD','VTI','SPY','XLF','XLU','XLV','VAW','VNQ','XLP','XLY','FXN','XLI','IYC','QQQ','BND','DJP','GLD','SHV']   # <-- Add tickers here

tickers_and_data_df = pd.DataFrame({})

for ticker in list_of_tickers_for_yahoo_data_dump:
    
    single_ticker_df = get_df_from_yahoo_finance(ticker)
    tickers_and_data_df = pd.concat([tickers_and_data_df, single_ticker_df])


In [33]:
#import fed rates on long term bonds (10 year)
long_term_fed_rates_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/Y10YDUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import consumer price index (inflation)
cpi_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/CPIUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import consumer confidence index
consumer_confidence_df = pd.read_csv(                    
    'https://www.econdb.com/api/series/CONFUS/?format=csv',                    
    index_col='Date', parse_dates=['Date'])

#import M2 supply (all money circulating and in bank accounts)
money_supply_df = pd.read_csv(                    
'https://www.econdb.com/api/series/M3US/?format=csv',                    
index_col='Date', parse_dates=['Date'])

In [34]:
#reset index
cpi_df= cpi_df.reset_index()
consumer_confidence_df =consumer_confidence_df.reset_index()
long_term_fed_rates_df= long_term_fed_rates_df.reset_index()
money_supply_df= money_supply_df.reset_index()

In [35]:
#convert date to datetime
cpi_df['Date'] = pd.to_datetime(cpi_df['Date'])
consumer_confidence_df['Date'] = pd.to_datetime(consumer_confidence_df['Date'])
long_term_fed_rates_df['Date'] = pd.to_datetime(long_term_fed_rates_df['Date'])
money_supply_df['Date'] = pd.to_datetime(money_supply_df['Date'])

In [36]:
#drop time from datetime format
cpi_df['Date'] = cpi_df['Date'].dt.date
consumer_confidence_df['Date'] = consumer_confidence_df['Date'].dt.date
long_term_fed_rates_df['Date'] = long_term_fed_rates_df['Date'].dt.date
money_supply_df['Date'] = money_supply_df['Date'].dt.date

In [37]:
#add columns to imitate yahoo df
cpi_df['Ticker'] = 'CPI'
cpi_df['Open'] = cpi_df['CPIUS']
cpi_df['High'] =cpi_df['CPIUS']
cpi_df['Low'] = cpi_df['CPIUS']
cpi_df['Close'] =cpi_df['CPIUS']
cpi_df['Adj Close'] = cpi_df['CPIUS']
cpi_df['Volume'] =cpi_df['CPIUS']

consumer_confidence_df['Ticker'] = 'CC'
consumer_confidence_df['Open'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['High'] =consumer_confidence_df['CONFUS']
consumer_confidence_df['Low'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['Close'] =consumer_confidence_df['CONFUS']
consumer_confidence_df['Adj Close'] = consumer_confidence_df['CONFUS']
consumer_confidence_df['Volume'] =consumer_confidence_df['CONFUS']

long_term_fed_rates_df['Ticker'] = 'Fed_Rates'
long_term_fed_rates_df['Open'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['High'] =long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Low'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Close'] =long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Adj Close'] = long_term_fed_rates_df['Y10YDUS']
long_term_fed_rates_df['Volume'] =long_term_fed_rates_df['Y10YDUS']

money_supply_df['Ticker'] = 'M3'
money_supply_df['Open'] = money_supply_df['M3US']
money_supply_df['High'] =money_supply_df['M3US']
money_supply_df['Low'] = money_supply_df['M3US']
money_supply_df['Close'] =money_supply_df['M3US']
money_supply_df['Adj Close'] = money_supply_df['M3US']
money_supply_df['Volume'] =money_supply_df['M3US']


In [38]:
#concatenate and convert all date column to datetime (again becuase it needs to be reminded)
test_df = pd.concat([tickers_and_data_df, cpi_df,consumer_confidence_df,long_term_fed_rates_df,money_supply_df], axis=0)
test_df.drop("CPIUS", axis=1, inplace=True)
test_df['Date']=pd.to_datetime(test_df['Date'])

In [39]:
#create pivot table
pivot = pd.pivot_table(
    data=test_df,
    index='Date',
    columns ='Ticker',
    
)

pivot =pivot.dropna()

In [40]:
# drop all columns except close
pivot = pivot['Close']


## Essential variables


In [41]:
# theres are the essentail variables

pct_change = pivot.pct_change().dropna()

corr_df = pct_change.corr()

list_of_tickers = ['BTC-USD','SPY','QQQ']

pct_change

Ticker,BND,BTC-USD,CC,CPI,DJP,FXN,Fed_Rates,GLD,IYC,M3,...,SPY,VAW,VNQ,VTI,XLF,XLI,XLP,XLU,XLV,XLY
Date,,,,,,,,,,,,,,,,,,,,,
2014-11-01,0.006192,0.117421,0.021818,-0.001685,-0.042278,-0.111570,0.013043,-0.004882,0.063391,0.006586,...,0.027472,0.010701,0.020003,0.024803,0.023490,0.030792,0.055449,0.011656,0.034785,0.054479
2014-12-01,-0.006034,-0.153034,0.054009,-0.002954,-0.083078,-0.026512,-0.051502,0.013112,0.015106,0.012484,...,-0.008012,-0.019806,0.005337,-0.005628,0.013525,-0.005799,-0.016430,0.026522,-0.017670,0.004735
2015-01-01,0.024038,-0.320835,0.048659,-0.006771,-0.044801,-0.043956,-0.149321,0.086899,-0.020616,-0.000085,...,-0.029629,-0.027097,0.068519,-0.027359,-0.069551,-0.035525,-0.009693,0.023295,0.013016,-0.029938
2015-02-01,-0.015056,0.169219,-0.028409,0.002556,0.028001,0.093453,0.053191,-0.059052,0.070709,0.007484,...,0.056205,0.077527,-0.036742,0.057420,0.058236,0.053509,0.041441,-0.063949,0.042876,0.085441
2015-03-01,0.003491,-0.039483,-0.024366,0.002975,-0.055839,-0.035649,0.030303,-0.021522,-0.001800,0.009285,...,-0.020080,-0.035797,0.011275,-0.016236,-0.009856,-0.029918,-0.025395,-0.017687,0.003599,-0.008161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-01,-0.020649,-0.168947,-0.048183,0.006426,0.101742,0.129195,0.197279,-0.016788,-0.096905,-0.000369,...,-0.052741,-0.070449,-0.084217,-0.060595,0.000256,-0.047916,-0.014784,-0.032551,-0.068564,-0.095334
2022-02-01,-0.013012,0.122394,-0.065422,0.008159,0.073371,0.067608,0.096591,0.061217,-0.030187,-0.000647,...,-0.029517,0.006120,-0.034827,-0.024867,-0.013825,-0.008438,-0.014084,-0.019061,-0.009678,-0.040660
2022-03-01,-0.029053,0.054301,-0.054166,0.012315,0.087507,0.108559,0.103627,0.012726,0.019981,0.013591,...,0.034377,0.053875,0.056856,0.029389,-0.005192,0.030934,0.013218,0.096128,0.054094,0.042665


# Start Mckenzie Test


In [42]:
#Calculate sharpe ratios

# sharpe_dict = {}

def sharpe_ratio_calculator(list):
    sharpe_dict = {}
    std = pct_change.std()
    for ticker in list:
        annualized_std = std[ticker]*np.sqrt(12)
        average_annual_return = pct_change[ticker].mean()*12
        sharpe_ratio = average_annual_return/annualized_std
        print(f"{ticker} sharpe ratio = {sharpe_ratio}")
        sharpe_dict.update({ticker:sharpe_ratio})
    return sharpe_dict

In [43]:
sharpe_dict=    sharpe_ratio_calculator(list_of_tickers_for_yahoo_data_dump)
sharpe_dict

BTC-USD sharpe ratio = 1.1328361597762027
VTI sharpe ratio = 0.6713796537130778
SPY sharpe ratio = 0.7158454722662906
XLF sharpe ratio = 0.5082097357186008
XLU sharpe ratio = 0.5196199071264465
XLV sharpe ratio = 0.6863684875633299
VAW sharpe ratio = 0.4760481547447656
VNQ sharpe ratio = 0.26520670973467725
XLP sharpe ratio = 0.5560973470956141
XLY sharpe ratio = 0.6881772685412784
FXN sharpe ratio = 0.10916771927292558
XLI sharpe ratio = 0.48498743272700273
IYC sharpe ratio = 0.6197435879609918
QQQ sharpe ratio = 0.9167464046768655
BND sharpe ratio = -0.21958608394766177
DJP sharpe ratio = 0.200134768916182
GLD sharpe ratio = 0.4601769104440686
SHV sharpe ratio = -0.014928532135956215


{'BTC-USD': 1.1328361597762027,
 'VTI': 0.6713796537130778,
 'SPY': 0.7158454722662906,
 'XLF': 0.5082097357186008,
 'XLU': 0.5196199071264465,
 'XLV': 0.6863684875633299,
 'VAW': 0.4760481547447656,
 'VNQ': 0.26520670973467725,
 'XLP': 0.5560973470956141,
 'XLY': 0.6881772685412784,
 'FXN': 0.10916771927292558,
 'XLI': 0.48498743272700273,
 'IYC': 0.6197435879609918,
 'QQQ': 0.9167464046768655,
 'BND': -0.21958608394766177,
 'DJP': 0.200134768916182,
 'GLD': 0.4601769104440686,
 'SHV': -0.014928532135956215}

In [46]:
# mckenzie test updated
def mckenzie_test(list_of_tickers, new_potential_holding, sharpe_dict, corr_df):
    sharpe_sum = 0
    corr_sum = 0
    for ticker in list_of_tickers:
        if ticker in sharpe_dict:
            sharpe_sum = sharpe_dict[ticker] + sharpe_sum
    for ticker in list_of_tickers:
        corr_sum += corr_df.loc[ticker,new_potential_holding]
    corr_average =corr_sum/len(list_of_tickers)
    sharpe_ratio_average_of_portfolio = sharpe_sum/len(list_of_tickers)
    if sharpe_dict[new_potential_holding] > (sharpe_ratio_average_of_portfolio * corr_average):
        # return "yes"
        print(f"{new_potential_holding} passes the McKenzie test and should be evaluated futher for your portfolio.")
        #print(f"The sharpe ratio of your current holding {current_holding} is {sharpe_dict[current_holding]} and {sharpe_dict[new_potential_holding]} for your new potential holding")
        #print(f"The expected correlation is {corr_df.loc[current_holding,new_potential_holding]}")
        #print(f"So therefore {corr_df.loc[current_holding,new_potential_holding]} * {sharpe_dict[current_holding]} is < {sharpe_dict[new_potential_holding]} so you should this passes the test")
        
    else:
        #return "no"
        print(f"{new_potential_holding} does not pass the McKenzie test")

In [47]:
mckenzie_test(list_of_tickers, 'BND', sharpe_dict, corr_df)



BND does not pass the McKenzie test


# If you want to test all the tickers against your portfolio

In [48]:
def test_all_tickers(list):
    for ticker in list:
        mckenzie_test(list_of_tickers,ticker,sharpe_dict, corr_df)

In [49]:
test_all_tickers(list_of_tickers_for_yahoo_data_dump)

BTC-USD passes the McKenzie test and should be evaluated futher for your portfolio.
VTI does not pass the McKenzie test
SPY passes the McKenzie test and should be evaluated futher for your portfolio.
XLF does not pass the McKenzie test
XLU passes the McKenzie test and should be evaluated futher for your portfolio.
XLV passes the McKenzie test and should be evaluated futher for your portfolio.
VAW does not pass the McKenzie test
VNQ does not pass the McKenzie test
XLP passes the McKenzie test and should be evaluated futher for your portfolio.
XLY passes the McKenzie test and should be evaluated futher for your portfolio.
FXN does not pass the McKenzie test
XLI does not pass the McKenzie test
IYC does not pass the McKenzie test
QQQ passes the McKenzie test and should be evaluated futher for your portfolio.
BND does not pass the McKenzie test
DJP does not pass the McKenzie test
GLD passes the McKenzie test and should be evaluated futher for your portfolio.
SHV passes the McKenzie test and